In [1]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import pandas as pd
import pymysql

model_path = "bert-base-chinese-finetuning-financial-news-sentiment"

model=AutoModel.from_pretrained(model_path)
tokenizer=AutoTokenizer.from_pretrained(model_path)
pipe = pipeline("text-classification", model=model_path,device=0)

conn = pymysql.connect(host='localhost', user='root', password='123456', database='citicup', charset='utf8')

# 创建游标
cursor = conn.cursor()

In [2]:
def getScore(input):
    response = pipe(input)
    score = 0
    weights = [0.6, 0.3, 0.1]  # 定义权重列表
    for index, i in enumerate(response):
        weight = weights[index] if index < len(weights) else 0  # 如果索引超出权重列表的长度，则权重为0
        if i['label'] == 'Positive':
            score += 1 * weight
        elif i['label'] == 'Negative':
            score += -1 * weight
        # print(i['label'], score)
    return score

In [3]:
df = pd.read_excel('ClimbingTitle.xls')
inputs = []

insert_sql = "INSERT INTO title_score (stock_code, score) VALUES (%s, %s)"

data_to_insert = []

for i, row in df.iterrows():
    inputs.append(row['post_title'])
    if (i + 1) % 3 == 0:
        stock_code = str(row['stock_code']).zfill(6) 
        score = getScore(inputs)
        score = "{:.1f}".format(score)
        data_to_insert.append((stock_code, score))
        inputs = []
        if len(data_to_insert) >= 1000:
            cursor.executemany(insert_sql, data_to_insert)
            conn.commit()
            data_to_insert = []

# 插入剩余的数据
if data_to_insert:
    cursor.executemany(insert_sql, data_to_insert)
    conn.commit()

cursor.close()
conn.close()

d:\anaconda\envs\test\lib\site-packages\transformers\pipelines\base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
